In [1]:
%load_ext autoreload
%autoreload 2

---

# SQLAlchemy

In [2]:
import sqlalchemy as sql

In [3]:
sql.__version__

'2.0.31'

In [4]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///:memory:", echo=True)

In [5]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [6]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [7]:
user_table.c.keys()

['id', 'name', 'fullname']

In [8]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [9]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [12]:
metadata_obj.drop_all(engine)

2024-07-04 12:39:41,784 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:39:41,785 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-07-04 12:39:41,786 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:41,787 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-07-04 12:39:41,788 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:41,789 INFO sqlalchemy.engine.Engine 
DROP TABLE address
2024-07-04 12:39:41,789 INFO sqlalchemy.engine.Engine [no key 0.00077s] ()
2024-07-04 12:39:41,790 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2024-07-04 12:39:41,791 INFO sqlalchemy.engine.Engine [no key 0.00092s] ()
2024-07-04 12:39:41,792 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
metadata_obj.create_all(engine)

2024-07-04 12:39:48,936 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:39:48,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-07-04 12:39:48,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:48,939 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-07-04 12:39:48,940 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:48,940 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-07-04 12:39:48,941 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:48,942 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-07-04 12:39:48,942 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 12:39:48,943 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-07-04 12:39:48,943 INFO sqlalchemy.engine.Engine [no key 0.00041s] ()
2024-07-04 12:39:48,944 INFO sqlalchemy.engine.Engine 
C

In [18]:
from sqlalchemy import insert
stmt = insert(user_table).values(id=2, name="spongebob", fullname="Spongebob Squarepants")

In [19]:
print(stmt)

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [20]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2024-07-04 12:41:49,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:41:49,873 INFO sqlalchemy.engine.Engine INSERT INTO user_account (id, name, fullname) VALUES (?, ?, ?)
2024-07-04 12:41:49,873 INFO sqlalchemy.engine.Engine [generated in 0.00170s] (2, 'spongebob', 'Spongebob Squarepants')
2024-07-04 12:41:49,874 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
result.inserted_primary_key

(2,)

In [23]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {'id': 3, "name": "sandy", "fullname": "Sandy Cheeks"},
            {'id': 4, "name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2024-07-04 12:47:19,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:47:19,016 INFO sqlalchemy.engine.Engine INSERT INTO user_account (id, name, fullname) VALUES (?, ?, ?)
2024-07-04 12:47:19,017 INFO sqlalchemy.engine.Engine [cached since 15.82s ago] [(3, 'sandy', 'Sandy Cheeks'), (4, 'patrick', 'Patrick Star')]
2024-07-04 12:47:19,018 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2024-07-04 12:54:30,951 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 12:54:30,952 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2024-07-04 12:54:30,953 INFO sqlalchemy.engine.Engine [generated in 0.00153s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2024-07-04 12:54:30,954 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
stmt = select(user_table).where(user_table.c.name == "spongebob")

In [28]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row.id)

2024-07-04 13:06:33,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 13:06:33,304 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-07-04 13:06:33,305 INFO sqlalchemy.engine.Engine [cached since 30.57s ago] ('spongebob',)
2
2024-07-04 13:06:33,305 INFO sqlalchemy.engine.Engine ROLLBACK


In [29]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [30]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


In [31]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address
